## Build and Save the Model

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore") 

data = pd.read_csv('batdongsandanang_data_cleaned.csv')
data.head()

,Title,Price(Ty),Area(m2),Bedroom,Toilet,Location,Price_per_m2(trieu)
0,Bán đất biển MT Hoàng Sa 205m2 - Đối diện bãi ...,0.0,205,NaN,NaN,"Sơn Trà, Đà Nẵng",NaN
1,Duy nhất lô góc ngã 4 shophouse Infinity 116m2...,0.0,116,NaN,NaN,"Liên Chiểu, Đà Nẵng",NaN
2,Bán gấp lô đất MT Thủ Khoa Huân - Gần Nguyễn V...,0.0,174,NaN,NaN,"Sơn Trà, Đà Nẵng",NaN
3,"Bán nhà An Mỹ 4, An Hải Tây, Sơn Trà, Đà Nẵng",4.3,56,2.0,2.0,"Sơn Trà, Đà Nẵng",76.79
4,Chuyển nhượng quỹ đất phát triển dự án trên 10...,0.0,5.000,NaN,NaN,"Điện Bàn, Quảng Nam",NaN


In [23]:
warnings.filterwarnings("ignore") 
data = pd.read_csv('batdongsandanang_data_cleaned.csv')

def clean_numeric_column(column):
    if data[column].dtype != 'object':
        data[column] = data[column].astype(str)
    data[column] = data[column].str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
    data[column] = pd.to_numeric(data[column], errors='coerce')

clean_numeric_column('Price_per_m2(trieu)')
clean_numeric_column('Area(m2)')

data.fillna(data.mean(numeric_only=True), inplace=True)

X = data.drop('Price(Ty)', axis=1)
y = data['Price(Ty)']

In [24]:
categorical_cols = ['Title', 'Location']  
numerical_cols = ['Area(m2)', 'Bedroom', 'Toilet', 'Price_per_m2(trieu)']

In [25]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ])

model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=123))  
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [26]:
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 504.17719923557905


In [33]:
import joblib

joblib.dump(model, 'bds_danang.joblib')

['bds_danang.joblib']

## Check the Model and Version of Scikit-Learn

In [40]:
import joblib
import pandas as pd

loaded_model = joblib.load('bds_danang.joblib')

input_data = {
    'Title': 'Bán nhà An Mỹ 4, An Hải Tây, Sơn Trà, Đà Nẵng', 
    'Area(m2)': 150,  
    'Bedroom': 3, 
    'Toilet': 2,  
    'Location': 'Sơn Trà, Đà Nẵng',  
    'Price_per_m2(trieu)': 70  
}

input_df = pd.DataFrame([input_data])

predictions = loaded_model.predict(input_df)

print(predictions)


[10.4476]


In [42]:
formatted_predictions = ["{:,.0f} tỷ".format(prediction) for prediction in predictions]

print(f"Dự đoán giá nhà (tỷ đồng): {formatted_predictions[0]}")

Dự đoán giá nhà (tỷ đồng): 10 tỷ


In [43]:
import sklearn
print(sklearn.__version__)

1.5.0


## Extract Useful Data

In [44]:
import pandas as pd

data = pd.read_csv('batdongsandanang_data_cleaned.csv')

print("Columns in the dataset:", data.columns)

data['Price(Ty)'] = pd.to_numeric(data['Price(Ty)'], errors='coerce')
data['Area(m2)'] = pd.to_numeric(data['Area(m2)'], errors='coerce')
data['Bedroom'] = pd.to_numeric(data['Bedroom'], errors='coerce')
data['Toilet'] = pd.to_numeric(data['Toilet'], errors='coerce')
data['Price_per_m2(trieu)'] = pd.to_numeric(data['Price_per_m2(trieu)'], errors='coerce')

data.fillna({
    'Price(Ty)': data['Price(Ty)'].mean(),
    'Area(m2)': data['Area(m2)'].mean(),
    'Bedroom': data['Bedroom'].mean(),
    'Toilet': data['Toilet'].mean(),
    'Price_per_m2(trieu)': data['Price_per_m2(trieu)'].mean()
}, inplace=True)

average_price = data['Price(Ty)'].mean()
average_area = data['Area(m2)'].mean()
average_bedroom = data['Bedroom'].mean()
average_toilet = data['Toilet'].mean()
average_price_per_m2 = data['Price_per_m2(trieu)'].mean()

print(f"Giá trung bình: {average_price:.2f} tỷ đồng")
print(f"Diện tích trung bình: {average_area:.2f} m²")
print(f"Số phòng ngủ trung bình: {average_bedroom:.2f}")
print(f"Số phòng tắm trung bình: {average_toilet:.2f}")
print(f"Giá mỗi mét vuông trung bình: {average_price_per_m2:.2f} triệu đồng")


Columns in the dataset: Index(['Title', 'Price(Ty)', 'Area(m2)', 'Bedroom', 'Toilet', 'Location',
       'Price_per_m2(trieu)'],
      dtype='object')
Giá trung bình: 10.95 tỷ đồng
Diện tích trung bình: 138.51 m²
Số phòng ngủ trung bình: 4.20
Số phòng tắm trung bình: 4.01
Giá mỗi mét vuông trung bình: 83.01 triệu đồng
